In [1]:
import pandas as pd
from openai import OpenAI

# Analyse der Stichwörter

Dieser Abschnitt befasst sich mit der Generierung von Stichwörtern zu den einzelnen Artikeln. Da NTV keine Stichwörter zu den Inhalten in ihren Artikeln mitliefert, müssen diese zunächst erstellt werden. Anschließend können Zusammenhänge und Themen in einem Graphen dargestellt werden. Zunächst wird der OpenAI-Client erstellt, über den anschließend die Abfragen gesendet werden und die CSV-Datei mit den Artikeln eingelesen. Ebenso wird die Spalte `keywords` hinzugefügt, die zunächst mit `None`-Werten gefüllt wird.

In [17]:
openai_client = OpenAI()

In [18]:
articles_data = pd.read_csv('2-1-data.csv')
articles = pd.DataFrame(articles_data)
articles = articles.drop('filename', axis=1)
articles["keywords"] = None
articles.head()

,date,time,category,headline,text,keywords
0,2024-05-10,17:14:00,politik,Gruppen treten am Vatertag mit Nazi Symbolik auf,Am Vatertag steuern Männer ein altes Militärfa...,None
1,2024-05-10,17:16:00,politik,Peking schimpft ueber US Zerstoerer im Suedchi...,China beansprucht die Paracel-Inseln im Südchi...,None
2,2024-05-10,19:33:00,politik,SPD Politiker Ecke schildert Sekunden der Prue...,Der vor einer Woche brutal attackierte SPD-Pol...,None
3,2024-05-10,08:40:00,politik,Landkreise fordern haertere Buergergeld Sankti...,Beim Bürgergeld in seiner jetzigen Form bemäng...,None
4,2024-05-10,15:44:00,politik,Buschmanns Regierungsflieger kehrt in Parkposi...,Deutsche Regierungsflieger sind für ihre Panne...,None


## Generierung der Stichwörter

Die Stichwörter zu den Artikeln sollen mit Hilfe von LLMs generiert werden, in diesem Fall mit Hilfe von ChatGPT. Verwendet wird, vorrangig aus kostengründen, das Modell `gpt-4o-mini`.

In [22]:
def create_keywords_for_articles(openai_client, articles_df, start=0, end=5):
    for i in range(start, end):
        results = []
        article_text = articles_df.iloc[i]["text"]
        article_headline = articles_df.iloc[i]["headline"]
        completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "Du fasst Nachrichtenartikel des Nachrichtensenders NTV anhand von Stichpunkten zusammenzufassen. Du gibst ausschließlich die Stichwörter durch Kommata getrennt zurück. Die Stichwörter sollen den Inhalt mit übergeordneten Kategorien wie Politik, Wirtschaft, Gesellschaft, Auto, Sport, Unterhaltung und dergleichen beschreiben. Es sollen exakt 10 Stichwörter generiert werden.",
            },
            {
                "role": "user",
                "content": article_text,
                },
            ],
        )
        keywords = completion.choices[0].message.content
        articles_df.at[i, "keywords"] = keywords
    return articles_df

In [23]:
articles = create_keywords_for_articles(openai_client, articles, 0, 100)
articles.head()

,date,time,category,headline,text,keywords
0,2024-05-10,17:14:00,politik,Gruppen treten am Vatertag mit Nazi Symbolik auf,Am Vatertag steuern Männer ein altes Militärfa...,"Politik, Gesellschaft, Rechtsradikalismus, Gew..."
1,2024-05-10,17:16:00,politik,Peking schimpft ueber US Zerstoerer im Suedchi...,China beansprucht die Paracel-Inseln im Südchi...,"China, USA, Südchinesisches Meer, Souveränität..."
2,2024-05-10,19:33:00,politik,SPD Politiker Ecke schildert Sekunden der Prue...,Der vor einer Woche brutal attackierte SPD-Pol...,"Politik, Gewalt, SPD, Matthias Ecke, Wahlkampf..."
3,2024-05-10,08:40:00,politik,Landkreise fordern haertere Buergergeld Sankti...,Beim Bürgergeld in seiner jetzigen Form bemäng...,"Bürgergeld, Landkreistag, Regelungen, Vermögen..."
4,2024-05-10,15:44:00,politik,Buschmanns Regierungsflieger kehrt in Parkposi...,Deutsche Regierungsflieger sind für ihre Panne...,"Regierungsflieger, Pannen, Technikprobleme, Rü..."


In [24]:
for i in range(100):
    print(articles.iloc[i]["keywords"])

Politik, Gesellschaft, Rechtsradikalismus, Gewalt, Symbolik, Polizei, Ermittlungen, Sachsen, Himmelfahrtstag, Straftaten
China, USA, Südchinesisches Meer, Souveränität, Militärübung, Taiwan, Paracel-Inseln, Konflikt, Sicherheit, Diplomatie
Politik, Gewalt, SPD, Matthias Ecke, Wahlkampf, Gesellschaft, Sicherheit, AfD, Extremismus, Erfahrung
Bürgergeld, Landkreistag, Regelungen, Vermögen, Sanktionen, Arbeitsmarkt, CDU, Grundsicherung, Reform, Existenzminimum
Regierungsflieger, Pannen, Technikprobleme, Rückreise, G7-Justizminister, Künstliche Intelligenz, Justizpolitik, Ersatzmaschine, Deutschland, Behördenkontakte
Politik, Menschenrechte, UN, Gaza, Massengräber, Untersuchung, Völkerrecht, Israel, EU, Sicherheit
Steve Bannon, Berufungsantrag, Haftstrafe, Missachtung des Kongresses, Kapitol-Erstürmung, Donald Trump, Bundesberufungsgericht, US-Rechtssystem, politische Kontroversen, Rechtspopulismus
Hochschulproteste, USA, Israel, Gazastreifen, Polizei, Räumung, Studenten, Konflikt, Antisemi